# loading

In [2]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Start capturing webcam video
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make pose detection
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show the frame
        cv2.imshow('MediaPipe Pose Estimation', image)

        # Break on 'q' key press
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# realtime

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Webcam
cap = cv2.VideoCapture(0)

# Use MediaPipe Pose with segmentation enabled
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  enable_segmentation=True,
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Pose estimation
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Create white background image
        bg_color = (255, 255, 255)  # white background
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = bg_color

        # If segmentation mask is available
        if results.segmentation_mask is not None:
            condition = results.segmentation_mask > 0.1  # 0.1 = threshold for visibility

            # Combine image and background using mask
            output_image = np.where(condition[..., None], image, bg_image)
        else:
            output_image = image

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show result
        cv2.imshow('Pose with Background Removed', output_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# image

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Ask user for image path
img_path = input("Enter the full path of the image to remove background from: ").strip()

# Check if the file exists
if not os.path.exists(img_path):
    print("Image not found. Exiting.")
    exit()

# Load input image
image = cv2.imread(img_path)
if image is None:
    print("Failed to load image. Exiting.")
    exit()

# Convert to RGB for MediaPipe
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose

with mp_pose.Pose(static_image_mode=True,
                  model_complexity=2,
                  enable_segmentation=True,
                  min_detection_confidence=0.5) as pose:

    results = pose.process(rgb_image)

    if results.segmentation_mask is not None:
        # Use mask to separate person from background
        condition = results.segmentation_mask > 0.1

        # Set background color (white in this case)
        background_color = (255, 255, 255)
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = background_color

        # Apply mask to get result
        output_image = np.where(condition[..., None], image, bg_image)

        # Show and save output
        cv2.imshow("Background Removed", output_image)
        cv2.imwrite("output_no_bg.jpg", output_image)
        print("Saved as output_no_bg.jpg")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Could not generate segmentation mask.")


Enter the full path of the image to remove background from:  ag.jpg


Saved as output_no_bg.jpg


# manual

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

img_path = input("image: ").strip()
if not os.path.exists(img_path):
    print("Image not found")
    exit()

image = cv2.imread(img_path)
h, w = image.shape[:2]
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mp_pose = mp.solutions.pose
pose_landmarks = mp_pose.PoseLandmark

with mp_pose.Pose(static_image_mode=True,
                  model_complexity=2,
                  enable_segmentation=False,
                  min_detection_confidence=0.5) as pose:

    results = pose.process(rgb_image)

    if not results.pose_landmarks:
        print("not detected.")
        exit()

    full_body_indices = [
        pose_landmarks.NOSE,                
        pose_landmarks.LEFT_SHOULDER,       
        pose_landmarks.LEFT_HIP,            
        pose_landmarks.LEFT_KNEE,           
        pose_landmarks.LEFT_ANKLE,         
        pose_landmarks.LEFT_HEEL,           
        pose_landmarks.LEFT_FOOT_INDEX,     
        pose_landmarks.LEFT_HIP,            
        pose_landmarks.RIGHT_HIP,           
        pose_landmarks.RIGHT_KNEE,          
        pose_landmarks.RIGHT_ANKLE,         
        pose_landmarks.RIGHT_HEEL,          
        pose_landmarks.RIGHT_FOOT_INDEX,    
        pose_landmarks.RIGHT_HIP,           
        pose_landmarks.RIGHT_SHOULDER,      
        pose_landmarks.RIGHT_ELBOW,         
        pose_landmarks.RIGHT_WRIST,         
        pose_landmarks.RIGHT_INDEX,         
        pose_landmarks.RIGHT_SHOULDER,      
        pose_landmarks.LEFT_SHOULDER,       
        pose_landmarks.LEFT_ELBOW,         
        pose_landmarks.LEFT_WRIST,          
        pose_landmarks.LEFT_INDEX          
    ]

    polyline_pts = []
    for idx in full_body_indices:
        lm = results.pose_landmarks.landmark[idx]
        x, y = int(lm.x * w), int(lm.y * h)
        polyline_pts.append((x, y))

    polyline_pts = np.array(polyline_pts, dtype=np.int32)

    mask = np.zeros((h, w), dtype=np.uint8)

    # Step 1: Draw the pose polyline corridor (4cm = ~151 pixels)
    corridor_thickness = 170
    cv2.polylines(mask, [polyline_pts], isClosed=False, color=255, thickness=corridor_thickness)

    # Step 2: Add extended torso rectangle
    shoulder_left = results.pose_landmarks.landmark[11]
    shoulder_right = results.pose_landmarks.landmark[12]
    hip_left = results.pose_landmarks.landmark[23]
    hip_right = results.pose_landmarks.landmark[24]

    # Convert to pixel coordinates
    sl_x, sl_y = int(shoulder_left.x * w), int(shoulder_left.y * h)
    sr_x, sr_y = int(shoulder_right.x * w), int(shoulder_right.y * h)
    hl_x, hl_y = int(hip_left.x * w), int(hip_left.y * h)
    hr_x, hr_y = int(hip_right.x * w), int(hip_right.y * h)

    # Extend shoulder points upward by 2.5 cm (~95 pixels)
    cm_to_pixels = 37.8
    shift = int(2.5 * cm_to_pixels)
    sl_y -= shift
    sr_y -= shift

    torso_pts = np.array([[
        (sl_x, sl_y),
        (sr_x, sr_y),
        (hr_x, hr_y),
        (hl_x, hl_y)
    ]], dtype=np.int32)

    cv2.fillPoly(mask, torso_pts, 255)

    mask_3ch = cv2.merge([mask, mask, mask])

    output = np.where(mask_3ch == 255, image, 255)  # white background outside

    # Show and save result
    cv2.imshow("Pose Line + Extended Torso", output)
    cv2.imwrite("output_pose_and_extended_torso.jpg", output)
    print("saved as output_pose_and_extended_torso.jpg")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
